In [84]:
import tensorflow as tf
import numpy as np
import os
import cv2
import glob 

In [85]:
data_path = os.path.join(os.getcwd(),'unlabeled_data')
paths = glob.glob(data_path + '\**\*.png', recursive = True)

In [86]:
output_file_path = '../data/tfrecords/train.tfrecords'
#if (os.path.isdir("../data/tfrecords/"))

In [87]:
def load_image(path):
    img = cv2.imread(path)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = img.astype(np.float32)
    img = img.astype(np.uint8)
    #np.uint8
    return img

In [88]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _floats_feature(value):
    return tf.train.Feature(
               float_list=tf.train.FloatList(value=value)
           )

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [89]:
#tf.io.TFRecordWriter
counter = 0
with tf.io.TFRecordWriter(output_file_path) as writer:
        for index, value in enumerate(paths):
            img = load_image(paths[index])
            image_h = img.shape[0]
            image_w = img.shape[1]
            if (image_h != 50) | (image_w != 50):
                continue
            counter += 1
            example = tf.train.Example(
              features=tf.train.Features(
                  #feature = get_feature(img)
                  feature = {
                      'height': _int64_feature(image_h),
                      'width': _int64_feature(image_w),
                      'image_raw': _bytes_feature(img.tostring())}
            ))
            writer.write(example.SerializeToString())
            print('\r{:.1%}'.format((index+1)/len(paths)), end='')
print("\ncounter :", counter)

100.0%
counter : 16194


In [68]:
data_path = os.path.join(os.getcwd(),'stl10_binary')
data_path = os.path.join(data_path,'unlabeled_X.bin')

In [71]:
def read_stl10_dataset(path_to_data):
    """
    :param path_to_data: the file containing the binary images from the STL-10 dataset
    :return: an array containing all the images
    """

    with open(path_to_data, 'rb') as f:
        # read whole file in uint8 chunks
        everything = np.fromfile(f, dtype=np.uint8)
        print("everything : ", type(everything))
        print("everything.shape : ", everything.shape)
        # We force the data into 3x96x96 chunks, since the
        # images are stored in "column-major order", meaning
        # that "the first 96*96 values are the red channel,
        # the next 96*96 are green, and the last are blue."
        # The -1 is since the size of the pictures depends
        # on the input file, and this way numpy determines
        # the size on its own.

        images = np.reshape(everything, (-1, 3, 96, 96))
        print("images.shape : ", images.shape)
        # Now transpose the images into a standard image format
        # readable by, for example, matplotlib.imshow
        # You might want to comment this line or reverse the shuffle
        # if you will use a learning algorithm like CNN, since they like
        # their channels separated.
        images = np.transpose(images, (0, 3, 2, 1))
        print("images.shape : ", images.shape)
        return images

In [72]:
images = read_stl10_dataset(data_path)
print(images.shape)

everything :  <class 'numpy.ndarray'>
everything.shape :  (2764800000,)
images.shape :  (100000, 3, 96, 96)
images.shape :  (100000, 96, 96, 3)
(100000, 96, 96, 3)


In [40]:
TRAIN_DATASET_DIR="../data/tfrecords/"
if not os.path.exists(TRAIN_DATASET_DIR):
    os.mkdir(TRAIN_DATASET_DIR)
    
TRAIN_FILE = 'train.tfrecords'
writer = tf.io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TRAIN_FILE))

In [41]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [42]:
def create_tfrecord_dataset(images, writer):

    # create training tfrecord
    read_imgs_counter = 0
    for i, image in enumerate(images):
            
        read_imgs_counter += 1
        image_h = image.shape[0]
        image_w = image.shape[1]

        img_raw = image.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw)}))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [43]:
create_tfrecord_dataset(images, writer)

End of TfRecord. Total of image written: 100000
